## Einführung in das Programmieren mit Python

# 8. Strings bearbeiten mit regulären Ausdrücken

### Prüfungsanmeldung

* bis 15. Juli
* Prüfungstermin: __Mittwoch, 20. Juli, 18:00, Hörsaal 3__ (wie im SB@Home vermerkt)

* Mögliche Prüfungen

    * PO 2015:
    
       * Programmieren (benotet), 326877
       * Skriptsprachen/FSQ (unbenotet), 326891

    * PO 2012:

       * Programmieren (5 ECTS) 317516
       * Skriptsprachen (3 ECTS) 314784
       
* auf der Klausur vermerken, für welches Modul sie gewertet werden soll

## Wiederholung: Dateien und Exceptions
* Dateien öffnen mit `f = open("datei.txt", "r", encoding="utf-8")`
* Methoden des Dateiobjekts zum Lesen und Schreiben, z.B. `read()`, `write()`
* Iterieren über Dateiobjekt zum zeilenweisen Lesen
* Dateien immer schließen
* `with open(…) as variable:` Automatisches Schließen am Ende des With-Blocks
* `try … except … else … finally`: Behandlung von Exceptions
* Module `os`, `os.path`, `shutil`: Dateioperationen

### Funktionen und Exceptions: Beispiel

In [35]:
import string
from collections import Counter

def tokenize_file(filename):   # kann IOError werfen
    with open(filename, encoding="utf-8") as f:
        return f.read().strip(string.punctuation).split()
    
def wordcounts(filename):
    wc = Counter(tokenize_file(filename))
    if len(wc.keys()) < 10:
        raise ValueError("Zu wenig unterschiedliche Wörter für ordentliche Statistik")
    return wc
        
def _main():
    filenames = ["roman.txt", "missing.txt", "short.txt", "beispiel.txt"]
    for filename in filenames:
        try:
            counts = wordcounts(filename)
            print("Top 3 in ", filename, ":", counts.most_common(3))
        except IOError as e:
            print("FEHLER:", filename, "kann nicht gelesen werden:", e)
        except ValueError as e:
            print("WARNUNG: Überspringe ", filename, ":", e)
            
if __name__ == "__main__":
    _main()

Top 3 in  roman.txt : [('Roman.', 2), ('Leser', 1), ('Helden', 1)]
FEHLER: missing.txt kann nicht gelesen werden: [Errno 2] No such file or directory: 'missing.txt'
WARNUNG: Überspringe  short.txt : Zu wenig unterschiedliche Wörter für ordentliche Statistik
Top 3 in  beispiel.txt : [('und', 2), ('große', 1), ('ſ!', 1)]


### Reguläre Ausdrücke

* Minisprache zur Beschreibung von Klassen von Zeichenketten
* Typische Anwendungen: Suchen, Ersetzen, Aufteilen, Umstellen von Zeichenketten, Extrahieren von Informationen aus Zeichenketten
* Beispiel: Literaturangaben in der Digitalen Bibliothek:

> Achim von Arnim: Sämmtliche Werke. Band 16, Berlin 1846

* reguläre Ausdrücke werden (mit leichten Abweichungen) von allen modernen Programmiersprachen und von allen halbwegs leistungsfähigen Texteditoren unterstützt

<h3>Einfache Anfänge</h3>

In [8]:
import re  # Modul für Reguläre Ausdrücke
s = 'todo\n1. Verordnung 2. \n2. Befehl II. \n3. Verweigerung 2. \n4. Militär 101'
print(s)

todo
1. Verordnung 2. 
2. Befehl II. 
3. Verweigerung 2. 
4. Militär 101


In [2]:
re.findall("Ver",s) 

['Ver', 'Ver']

### Operationen mit Standardmitteln, ohne RE

In [3]:
# Zur Erinnerung: einfache Stringsuche
"welt" in "hallo welt"

True

In [4]:
#einfache ersetzung
"hallo welt".replace("welt", "du")

'hallo du'

### reguläre Ausdrücke anwenden (1)

`re.findall(pattern,string)` findet alle Treffer von `pattern` in `string`

In [5]:
import re
re.findall("ha","aha, das habe ich mir gedacht.")

['ha', 'ha']

`re.sub(pattern,replace,string)` – **sub**stitute

In [6]:
re.sub("ich","du","ich ich ich - schönes Wort")

'du du du - schönes Wort'

### Begrifflichkeiten

* Ein Regulärer Ausdruck heißt auch **Pattern**
* ein Pattern **matcht** auf einen String (oder eben auch nicht).
* `re.findall(pattern, string)` findet alle Teilstrings von `string`, auf die `pattern` matcht.

### Einfache reguläre Ausdrücke

* »normale« Zeichen (= keine Metazeichen) matchen sich selbst. `a` matcht `a`.
    * _Metazeichen_ sind `. ^ $ * + ? { } [ ] \ | ( )`, sie können gematcht werden, indem man einen `\` voranstellt 
* Ein Punkt `.` matcht ein beliebiges Zeichen (außer Newline). `.` matcht `a` oder `b` oder …    
* Zusammensetzen = Hintereinanderschreiben.
    * `abc` matcht "abc"
    * `H. h. h.` matcht z.B. "Ha ha ha" oder "Ho ho ho" oder "He ha hi"

### Zeichenklassen

* `[abc]` matcht ein Zeichen, das `a` oder `b` oder `c` ist.
* `[A-Fa-f]` matcht einen der Groß- oder Kleinbuchstaben von A bis F
* `^` am Beginn einer Zeichenklasse invertiert die Klasse:
   * `[^0-9]` matcht jedes Zeichen, das _keine_ Ziffer von 0-9 ist
* In Zeichenklassen gelten Metazeichen nicht: `[.]` matcht ebenso wie `\.` einen Punkt

#### häufige Zeichenklassen

-   `.` jedes Zeichen außer neue Zeile (`\n`)
-   `\d` jede Ziffer, z.B. 1, 4, 0 <br/>
    `\D` jedes Zeichen, das **keine** Ziffer ist.
-   `\s` jedes whitespace-Zeichen, z.B. Leerzeichen, \n, \t<br/>
    `\S` jedes Zeichen, das **kein** whitespace ist.
-   `\w` »Identifier-Zeichen«, also Buchstaben, z.B. A g ö ß 4 é € α И, Ziffern, Unterstrich _ <br/>
    `\W` jedes Zeichen, das **kein** Identifier-Zeichen ist.

Die Definitionen beziehen sich auf die Unicode-Zeicheneigenschaften. Wer Zugriff auf die kompletten Unicode-Properties will, muss das externe Modul _regex_ installieren.

#### Raw Strings
* Um ein `\` zu matchen, benötigen Sie die RE `\\`, in einem Python-String `"\\\\"` ...
* in einem _raw string_ hat ``\`` keine Sonderbedeutung
* Syntax `r"…"` bzw. `r'…'`

In [32]:
print(r"C:\Windows", "Bla\nBlubb", r"Bla\nBlubb",)

C:\Windows Bla
Blubb Bla\nBlubb


<h3 style="color:green">Aufgaben</h3>
Gegeben ist der String: `s = "1. Ja. 2. Nein. 3. Gut. "`
<ol>
<li>Finden Sie alle Zahlen in dem String.</li>
<li>Finden Sie alle Großbuchstaben in dem String.</li>
<li>Finden Sie alle Leerzeichen in dem String.</li>
<li>Ersetzen Sie 'Ja' mit 'Yes', 'Nein' mit 'No' und 'Gut' mit 'ok'.</li>
</ol>

<h3>Musterlösung</h3>
<p>Gegeben ist der String: s = "1. Ja. 2. Nein. 3. Gut. "</p>

In [12]:
s = "1. Ja. 2. Nein. 3. Gut. "

<p>Finden Sie alle Zahlen in dem String.</p>

In [13]:
import re
re.findall(r"\d", s)

['1', '2', '3']

<p>Finden Sie alle Großbuchstaben in dem String.</p>

In [14]:
re.findall(r"[A-ZÄÖÜ]", s)

['J', 'N', 'G']

<p>Finden Sie alle Leerzeichen in dem String.</p>

In [15]:
re.findall(r"\s", s)

[' ', ' ', ' ', ' ', ' ', ' ']

<p>Ersetzen Sie 'Ja' mit 'Yes', 'Nein' mit 'No' und 'Gut' mit 'ok'.</p>

In [16]:
s = re.sub(r"Ja", "Yes", s)
s = re.sub(r"Nein", "No", s)
re.sub(r"Gut", "ok", s)

'1. Yes. 2. No. 3. ok. '

### Entwickeln regulärer Ausdrücke

Es gibt Tools, die Ihnen bei der Entwicklung komplexr regulärer Ausdrücke helfen, indem sie die Treffer (und Gruppen, s.u.) eines regulären Ausdrucks in einem Beispielstring direkt beim Bearbeiten des Patterns markieren, z.B.:

* [regex101.com](https://regex101.com/#python) oder [pythex.org](http://pythex.org/) im Web
* [redemo.py](https://hg.python.org/cpython/file/3.3/Tools/demo/redemo.py) als lokale Anwendung aus dem Python-Projekt selbst

### Wiederholungen
* `*` matcht auf **0 oder mehr** Wiederholungen des vorherigen Zeichens/Teilausdrucks
* `+` matcht auf **1 oder mehr** Wiederholungen des vorherigen Zeichens/Teilausdrucks
* `?` matcht auf **0 oder 1** Wiederholungen des vorherigen Zeichens/Teilausdrucks
* `{n,m}` matcht auf **n bis m** Wiederholungen des vorherigen Zeichens/Teilausdrucks 

In [17]:
s = "Ha HaHa Haahaa Hai Hi Ho"
print("*", re.findall("H[ai]*", s))
print("+", re.findall("H[ai]+", s))
print("?", re.findall("H[ai]?", s))
print("{2,3}", re.findall("H[ai]{2,3}", s))

* ['Ha', 'Ha', 'Ha', 'Haa', 'Hai', 'Hi', 'H']
+ ['Ha', 'Ha', 'Ha', 'Haa', 'Hai', 'Hi']
? ['Ha', 'Ha', 'Ha', 'Ha', 'Ha', 'Hi', 'H']
{2,3} ['Haa', 'Hai']


<h3 style="color:green">Aufgaben</h3>
<code>s = 'todo\n1. Verordnung 2. \n2. Befehl II. \n3. Verweigerung 2. \n4. Militär 101'</code>

1. Finden Sie alle (arabischen) Zahlen.
2. Suchen Sie in der Liste (ohne todo) alle Wörter, die 6 Buchstaben oder weniger haben

<h3>Musterlösung</h3>

<code>s = 'todo\n1. Verordnung 2. \n2. Befehl II. \n3. Verweigerung 2. \n4. Militär 101'</code>

(1) Finden Sie alle (arabischen) Zahlen.

In [16]:
re.findall('[0-9]+', s)

['1', '2', '2', '3', '2', '4', '101']

(2) Suchen Sie in der Liste (ohne todo) alle Wörter, die 6 Buchstaben oder weniger haben

In [17]:
s = 'todo\n1. Verordnung 2. \n2. Befehl II. \n3. Verweigerung 2. \n4. Militär 101'
re.findall(r" \w{1,6} ", s)

[' Befehl ']

<h3>Positionen / Anker</h3>
… matchen nicht _auf_, sondern _vor/nach/zwischen_ Zeichen
<ul>
<li>'^' matches the start of the string</li>
<li>'$' matches the end of the string</li>
<li>'\b' matches the empty string but only at the beginning or ending of a word <br/>'\B' matches the empty string but only if it is not at the beginning or ending of a word</li>
</ul>


In [18]:
s = "hallo welt! wie geht es dir?"
re.findall(r"^\w+", s)

['hallo']

In [19]:
re.findall(r"\bfoo\b", "foo. (foo) foobar")

['foo', 'foo']

### Gruppierungen

* Runde Klammern um einen Teilausdruck bilden eine **Gruppe**.
* Quantoren gelten für die ganze Gruppe -> `(bla)+` matcht `blablabla`
* Gruppen können separat referenziert werden

In [20]:
s = "Sehr geehrte Frau Mustermann,"
m = re.match("Sehr geehrter? (.*),", s)
m.groups()

('Frau Mustermann',)

Gruppen können mit `\1`, `\2`, ... auch innerhalb des Ausdrucks referenziert werden:


In [21]:
expr = "n = n + 1; test = test + n; n = test + 1"
simplified = re.sub(r"(\w+) = \1 \+ (\w+)",   # diese RE bildet zwei Gruppen, die in der RE (\1) …
                    r"\1 += \2",              # und im 'ersetzen'-String (\1, \2) referenziert werden
                    expr)
print(simplified)

n += 1; test += n; n = test + 1


<h3>Greedy vs. Non-Greedy</h3>
<p>Voreinstellung: greedy, d.h. das die größtmögliche Zeichenkette gesucht wird, die zum regulären Ausdruck passt.</p>
<p>Durch ? nach dem Quantifier Umstellung auf non-greedy</p>

In [22]:
re.findall("b.*t", "She booted the robot.")

['booted the robot']

In [23]:
re.findall("b.*?t", "She booted the robot.")

['boot', 'bot']

### Oder

Mit `|` können Sie nach Alternativen suchen:

In [1]:
for match in re.finditer(r"(Hai|Krokodil|Piranha)\w*", "Bond sollte schon durch Haibiss, Piranhas und Krokodile sterben."):
    print("Todesursache:", match.group(0))

NameError: name 're' is not defined

### Reguläre Ausdrücke anwenden (2)
![Übersicht über das re-Modul](images/re-uebersicht.svg)

* `re.compile` kompiliert einen regulären Ausdruck zu einem Objekt. Wenn Sie einen regulären Ausdruck oft benötigen (z.B. in einer Schleife), verwenden Sie diese Funktion (außerhalb der Schleife) und arbeiten mit dem Objekt, das sie zurückgibt: das ist schneller.
* _Match-Objekte_ beschreiben einen Treffer in einem String genauer: Sie liefern z.B. Zugriff auf die einzelnen Gruppen und auf die genaue Stelle im Suchstring, an der der Ausdruck gematcht hat.
* Die Funktionen/Methoden `match` und `search` liefern jeweils ein Match-Objekt: `match` matcht nur am _Beginn_ des Suchstrings, `search` findet den ersten Treffer (ggf. ab einem bestimmten Offset, lesen Sie dazu die Dokumentation zu den Funktionen). `finditer` liefert einen Iterator über alle Treffer, jeder Treffer wird als Matchobjekt zurückgegeben. Sie können darüber in einer `for`-Schleife iterieren:

In [25]:
for match in re.finditer("\d+", "Lesen Sie die Kapitel 3 und die Seiten 55-60"):
    print("An Position {} steht die Zahl {}.".format(match.start(), match.group(0)))

An Position 22 steht die Zahl 3.
An Position 39 steht die Zahl 55.
An Position 42 steht die Zahl 60.


* `split` spaltet einen String an einem gegebenen regulären Ausdruck auf, liefert also quasi das Komplement zu `findall`:

In [26]:
re.split(r'[,.;:!?]+\s*', 'Hier: Nimm ein paar Sätze! Zerlegst du sie mir?')

['Hier', 'Nimm ein paar Sätze', 'Zerlegst du sie mir', '']

* mit `escape` können Sie alle potentiellen Metazeichen in einem String escapen:

In [27]:
print(re.escape("Sonne, Mond [und] Sterne***"))

Sonne\,\ Mond\ \[und\]\ Sterne\*\*\*


<h3 style="color:green">Übungsaufgaben</h3>
<p>s = "120313130414300312" 
<ol>
<li>Angenommen es handelt sich hier um drei Datumsangaben in Folge im Format TTMMJJ. Trennen Sie die Angaben durch einen Bindestrich: 120313-130414-300312</li>
<li>Bearbeiten Sie den String weiter, so dass nun normale dt. Datumsangaben zu lesen sind: 12.03.13-13.04.14-30.03.12
<li>Bearbeiten Sie den String s weiter, so dass am Ende amerikanische Datumsangaben dastehen: 03/12/13-...</li>
</ol>

In [28]:
import re

s = "120313130414300312"

s1 = re.sub(r"(\d{6})\B", r"\1-", s)
print(s1)

120313-130414-300312


In [29]:
s2 = re.sub(r"(\d\d)(\d\d)(\d\d)", r"\1.\2.\3", s1)
print(s2)

12.03.13-13.04.14-30.03.12


In [31]:
s3 = re.sub(r"(\d\d)\.(\d\d)\.(\d\d)", r"\2/\1/\3", s2)
print(s3)

03/12/13-04/13/14-03/30/12
